In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep

# crawing in jetson nano by chromium
# driver = webdriver.Chrome()
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

# load this article
url = ""
driver.get(url)

# and wait a bit
sleep(7)

# get the text of the desired element
# click #usm-grade-post-ann-id > div.usm-post-ann-con > div
element = driver.find_element(By.CSS_SELECTOR, "#usm-grade-post-ann-id > div.usm-post-ann-con > div")

# get the text
text2 = element.text


# write the text to a file
with open("text2.txt", "w") as file:
    file.write(text2)

# it works, nice
driver.quit()

In [ ]:
from inputs import get_gamepad
import cv2
import datetime
import json
import base64
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import threading
from pydub import AudioSegment
from pydub.playback import play
import time
from Arm_Lib import Arm_Device
import os
from tkinter import *
from tkinter import simpledialog

pass_sound = AudioSegment.from_file("pass.mp3")
fail_sound = AudioSegment.from_file("fail.mp3")

Arm = Arm_Device()
# default_motor_angles = [90, 90, 14, 21, 90, 164]
# goal_motor_angles = [90, 68, 0, 37, 90, 164]
# timearc = 500  # 5 seconds

frame = None
text2 = None

def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

def process_frame(frame, threshold):
    global text2

    now = datetime.datetime.now()
    filename = now.strftime('%m%d%H%M%S') + '.jpg'
    cv2.imwrite(filename, frame)

    with open(filename, "rb") as f:
        img = base64.b64encode(f.read())

    URL = "https://uajh3ipk7g.apigw.ntruss.com/custom/v1/19624/e1163e7dd3558194ff77ee4240d6d88da8b100546c846b80595e8390f9f4ca65/general"
    KEY = ""

    headers = {
        "Content-Type": "application/json",
        "X-OCR-SECRET": KEY
    }

    data = {
        "version": "V2",
        "requestId": "sample_id",
        "timestamp": 0,
        "images": [
            {
                "name": "sample_image",
                "format": "png",
                "data": img.decode('utf-8')
            }
        ]
    }
    data = json.dumps(data)
    response = requests.post(URL, data=data, headers=headers)
    res = json.loads(response.text)

    a = []
    count = len(res["images"][0]["fields"])
    for i in range(count):
        texts = res["images"][0]["fields"][i]["inferText"]
        isget = res["images"][0]["fields"][i]["boundingPoly"]["vertices"][0]["y"]
        a.append([texts, isget])

    output = []
    current_line = ""
    for i in range(len(a) - 1):
        if abs(a[i][1] - a[i + 1][1]) < 10:
            current_line += a[i][0] + " "
        else:
            current_line += a[i][0]
            output.append(current_line)
            current_line = ""
    current_line += a[-1][0]
    output.append(current_line)

    print(output)

    text1 = "\n".join(output)

    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf[0:1], tfidf[1:2])
    print("Cosine similarity: ", similarity)

    jaccard_score = jaccard_similarity(text1, text2)
    print("Jaccard score: ", jaccard_score)

    new_filename = now.strftime('%m%d%H%M%S') + '_' + str(jaccard_score) + '.jpg'
    os.rename(filename, new_filename)

    if jaccard_score >= threshold:
        print("pass")
        play(pass_sound)
        Arm.Arm_serial_servo_write6(90,59,33,49,90,164, 500)
        time.sleep(1)
        Arm.Arm_serial_servo_write6(90,131,23,2,90,164, 500)
    else:
        print("fail")
        play(fail_sound)

def event_check(user_input):
    global frame
    while True:
        events = get_gamepad()
        for event in events:
            if event.code == 'BTN_TRIGGER' and event.state:
                if frame is not None:
                    process_frame(frame, user_input)
            elif event.code == 'BTN_THUMB2' and event.state:
                cv2.destroyAllWindows()
                print("Exiting...")
                exit()

def main():
    global frame
    global text2

    with open("text2.txt", "r") as file:
        text2 = file.read()

    Arm.Arm_serial_servo_write6(90,131,23,2,90,145, 500)

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    root = Tk()
    root.withdraw()

    user_input = simpledialog.askfloat("Input", "Enter a number between 0.1 and 1", minvalue=0.1, maxvalue=1.0)

    event_thread = threading.Thread(target=event_check, args=(user_input,))
    event_thread.start()

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        cv2.imshow('frame', frame)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()